In [65]:
import string 
import json 

import pandas as pd
import nltk
import spacy


from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
nlp = spacy.load('de_core_news_sm') 

#Definition für top keywords bzw. Ermittlung TF-IDF Werte
def get_top_n_kw(document, vectorizer, n):
    matrix = vectorizer.transform([document])
    scores = pd.DataFrame(matrix[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"])
    scores = scores[scores["TF-IDF"]!=0]
    return {k: v["TF-IDF"] for k,v in scores.sort_values("TF-IDF", ascending=False)[:n].to_dict("index").items()}
    


c:\Users\ddoro\.conda\envs\jemen-scraper\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\ddoro\.conda\envs\jemen-scraper\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\ddoro\.conda\envs\jemen-scraper\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\ddoro\.conda\envs\jemen-scraper\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\ddoro\.

In [66]:
#Preprocessing mit Lemmatisierung, Umlauten und stemming
with open ("../analysis/custom-replacements.json", "r") as file:
    replacements = json.load(file)

def preprocess(article):
    document = " ".join([article["text"], article["title"], article["surtitle"]]).lower()
    document.replace("sammads", "sammad")
    for sw in ["\n"]:
        document = document.replace(sw, " ")
    # part of speech tagging und lemmatization
    document = " ".join([t.lemma_ for t in nlp(document) if t.pos_ in ["NOUN", "PROPN", "VERB", "X"]]).lower()
    for k,v in replacements.items():
        document = document.replace(k, v)
    for n in range(10):
        document = document.replace(str(n), "")
    article["preprocessed_text"] = document 
    return article

In [67]:
# preprocess corpus and save preprocessed text in article (corpus is a list of article dictionaries)
with open("../data/sz/all-relevant-fulltext.json", "r", encoding="utf-8") as file:
    s = file.read()
    articles_sz = json.loads(s)

with open("../data/faz/all-relevant-fulltext.json", "r", encoding="utf-8") as file:
    s = file.read()
    articles_faz = json.loads(s)

corpus = [preprocess(article) for article in articles_faz + articles_sz]


In [68]:
# generate tfidf matrices using scikit learn vectorizer and generate n keywords for every article and save it in article dict
nltk.download('stopwords')
german_stop_words = stopwords.words('german')

tfidf_vect = TfidfVectorizer(stop_words=german_stop_words)
tfidf_vect = tfidf_vect.fit([article["preprocessed_text"] for article in corpus])

for article in corpus:
    article["keywords"] = get_top_n_kw(article["preprocessed_text"], tfidf_vect, 10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddoro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
# generate a list of all keywords occuring onces
final_keywords = []

for article in corpus:
    final_keywords += list(article["keywords"].keys())


print(f"A total of {len(final_keywords)} keywords were produced" )
unique_keywords = list(set(final_keywords))
print(f"An {len(unique_keywords)} unique ones")


A total of 6830 keywords were produced
An 3350 unique ones


In [70]:
# Liste von keywords die öfter als einmal vorkommen
from collections import Counter
c = Counter(final_keywords)
all_kw_with_frequency =[(x,v) for x,v in dict(c).items()]
keywords_that_occur_min_twice = {x:v for x,v in dict(c).items() if v>1}
singular_occurences = [x for x,v in dict(c).items() if v==1]

In [71]:
with open("tmp.txt", "w", encoding="utf-8") as f:
    json.dump(unique_keywords, f, indent=2, ensure_ascii=False)

In [72]:
# frequent itemset analyse 
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

import pandas as pd

def remove_singular_kws(kws, to_remove):
    return [kw for kw in kws if kw not in to_remove]

dataset = [remove_singular_kws(article["keywords"].keys(), singular_occurences) for article in corpus]
# dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
#            ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
#            ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
#            ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
#            ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']] 

te = TransactionEncoder()

te_ary = te.fit(dataset).transform(dataset)

df = pd.DataFrame(te_ary, columns=te.columns_)


frequent_itemsets = apriori(df, min_support=0.003, use_colnames=True)

frequent_itemsets.tail(10)


,support,itemsets
2867,0.004392,"(firma, motoren, herr, modellbau, hanau, tdqq,..."
2868,0.004392,"(firma, motoren, herr, modellbau, hanau, drohn..."
2869,0.004392,"(firma, herr, modellbau, hanau, tdqq, drohnen,..."
2870,0.004392,"(firma, motoren, herr, modellbau, hanau, tdqq,..."
2871,0.004392,"(firma, motoren, herr, hanau, tdqq, drohnen, w..."
2872,0.004392,"(firma, motoren, modellbau, hanau, tdqq, drohn..."
2873,0.004392,"(firma, motoren, herr, modellbau, tdqq, drohne..."
2874,0.004392,"(motoren, herr, modellbau, hanau, tdqq, drohne..."
2875,0.004392,"(firma, motoren, herr, modellbau, hanau, tdqq,..."
2876,0.004392,"(firma, motoren, herr, modellbau, hanau, tdqq,..."


In [73]:
# create dataframe w/ random sample of articles and respective keywords
import random
sample = [{"link":a["link"], "kws": ", ".join(a["keywords"].keys())} for a in random.sample(corpus, 30)]
pd.DataFrame(sample).to_csv("random_articles_for_categorie_gen.tsv", sep="\t")



In [74]:
corpus_df = pd.DataFrame(corpus)
years = list(set([(d.year) for d in corpus_df["date"]]))

samples_by_year = pd.DataFrame()

for year in years:
    tmp = pd.DataFrame([article for article in corpus if year == article["date"].year])
    print(len(samples_by_year))
    if not len(samples_by_year):
        samples_by_year = tmp.sample(10)
    else:
        samples_by_year = pd.concat([samples_by_year, tmp.sample(10)])

samples_by_year.to_csv("samples_by_year.tsv", sep="\t", index=False)

AttributeError: 'str' object has no attribute 'year'

In [ ]:
2018 is corpus[0]["date"].year

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\ddoro\AppData\Local\Temp\ipykernel_12428\1971795393.py:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  2018 is corpus[0]["date"].year


False

In [ ]:
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(25))

NameError: name 'tfIdf' is not defined

In [ ]:
get_values_with_feature_names(tfIdfVectorizer.transform([dataset[0]]), tfIdfVectorizer)


,TF-IDF
about,0.256860
and,0.256860
enjoy,0.195349
in,0.000000
is,0.256860
learning,0.513720
library,0.000000
machine,0.513720
my,0.256860
park,0.000000


Creating random sample for each year w/ 4 articles per year

In [ ]:
# create dataframe w/ random sample of articles and respective keywords
import random

for article in corpus:
    article['date'] = pd.to_datetime(article['date'], format="%d.%m.%Y")
    year = article['date'].year
    print(year)

#corpus['year'].groupby(article['year'].year)

sample_per_year = [{"link":a["link"], "date":a["date"], "kws": ", ".join(a["keywords"].keys())} for a in random.sample(corpus, 28)]
pandas.DataFrame(sample_per_year).groupby(year).groups

pd.DataFrame(sample_per_year).to_csv("random_articles_per_year_for_categorie_gen.tsv", sep="\t")



#     for article in articles_html:
#         title = article.select("span[class*='boxHeader']")[0].text.strip()
#         link = "https://www.wiso-net.de/" + article.find("a", {"rel":"nofollow"})["href"]
#         date = article.find("td", {"class":"boxCol3"}).text.strip()
#         abstract =  article.select("span[class*='boxDescription']")[0].text.strip() + " ".join([e.text.strip() for e in article.select("span[class*='boxAbstract']")[1:]])
#         author = ""
#         surtitle = "" 


#         articles_list.append({"title":title, "link":link, "date":date, "abstract":abstract, "author":author, "surtitle":surtitle, "source":source})
#     return articles_list


2018
2018
2017
2016
2016
2016
2016
2018
2020
2019
2019
2022
2017
2017
2017
2017
2017
2017
2017
2020
2020
2020
2020
2020
2022
2019
2019
2019
2019
2019
2019
2019
2019
2015
2019
2019
2019
2019
2017
2017
2017
2017
2017
2016
2017
2017
2017
2017
2017
2017
2017
2021
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2022
2022
2022
2022
2022
2022
2022
2022
2017
2017
2017
2017
2017
2017
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2018
2018
2018
2018
2018
2018
2019
2019
2019
2019
2019
2019
2019
2019
2019
2018
2018
2019
2021
2021
2021
2021
2021
2021
2018
2015
2015
2015
2015
2015
2018
2018
2018
2015
2015
2015
2015
2015
2015
2015
2015
2018
2018
2018
2018
2018
2021
2021
2021
2021
2021
2021
2021
2018
2018
2018
2018
2018
2018
2018
2015
2015
2015
2015
2015
2015
2015
2015
2015
2015
2019
2019
2019
2020
2020
2020
2020
2017
2021
2021
2021
2021
2021
2021
2020
2020
2019
2019
2019
2019
2019
2016
2016
2016
2016
2016
2016
2016


TypeError: list indices must be integers or slices, not str

In [92]:
with open ("corpus.json", "w") as file:
    json.dump(corpus, file, indent = 2)

